# Spatial coincidence assessed by MPP encodings

## Package setup

In [ ]:
import numpy as np
import scipy
from odyssey_geo.encoders import MPPEncoder

## Setup

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_array
from sklearn.cluster import DBSCAN
import shapely
import shapely.wkt
import plotly
import plotly.subplots
from plotly.graph_objs import Scatter, Figure

## Create an embedding for a domain

In [ ]:
x0, y0 = 0, 0
x1, y1 = 100, 100
resolution = 10
scale = 10
encoder = MPPEncoder(domain=[x0, y0, x1, y1], resolution=resolution, scale=scale, center=True)
print('%d reference points in encoder' % encoder.n_ref)

## Create geometries

In [ ]:
# linestrings.
xx = [10, 20, 35, 77, 90]
yy = [10, 25, 42, 67, 80]
xy = ', '.join(['%.1f %.1f' % (z) for z in list(zip(xx, yy))])
wkt = 'LINESTRING(%s)' % xy
ls1 = shapely.from_wkt(wkt)

xx = [15, 29, 46, 77, 85]
yy = [92, 73, 55, 29, 17]
xy = ', '.join(['%.1f %.1f' % (z) for z in list(zip(xx, yy))])
wkt = 'LINESTRING(%s)' % xy
ls2 = shapely.from_wkt(wkt)

ls1, ls2

In [ ]:
# For each linestring, generate multipoints in its general vicinity.
from scipy.interpolate import interp1d

def generate_multipoint(geom, n=20):
    points = []
    for d in np.linspace(0, geom.length, n):
        p = shapely.line_interpolate_point(geom, d)
        dx = (np.random.random() - 0.5) * 40
        dy = (np.random.random() - 0.5) * 40
        points.append(shapely.affinity.translate(p, xoff=dx, yoff=dy))
    mp = shapely.MultiPoint(points)
    return mp

mp1 = generate_multipoint(ls1, 20)
mp2 = generate_multipoint(ls2, 25)


In [ ]:
def draw_linestring(geom, color='gray', name='linestring'):
    coords = geom.coords
    xx = [z[0] for z in coords]
    yy = [z[1] for z in coords]
    trace = Scatter(
        x=xx, y=yy, name=name, 
        mode='lines', marker={'color': color, 'size': 12}
    )
    return trace

def draw_multipoint(geom, color='gray', name='multipoint'):
    xx = [z.xy[0][0] for z in geom.geoms]
    yy = [z.xy[1][0] for z in geom.geoms]
    trace = Scatter(
        x=xx, y=yy, name=name, 
        mode='markers', marker={'color': color, 'size': 12}
    )
    return trace

In [ ]:
fig = plotly.subplots.make_subplots(1, 1)

# Draw the reference points.
# trace = Scatter(
#     x=encoder.ref_x, y=encoder.ref_y, name='reference points', 
#     mode='markers', marker={'color': 'gray', 'size': 5, 'symbol': 'cross'}
# )
# fig.append_trace(trace, 1, 1)

# Add the linestrings.
fig.append_trace(draw_linestring(ls1, color='red', name='linestring 1'), 1, 1)
fig.append_trace(draw_linestring(ls2, color='blue', name='multipoint 1'), 1, 1)
fig.append_trace(draw_multipoint(mp1, color='green', name='linestring 2'), 1, 1)
fig.append_trace(draw_multipoint(mp2, color='magenta', name='multipoint 2'), 1, 1)


fig['layout']['width'] = 600
fig['layout']['height'] = 500
fig['layout']['xaxis1']['range'] = [0, 100]
fig['layout']['yaxis1']['range'] = [0, 100]

fig

In [ ]:
# Get a matrix of distances between embedding vectors for the four shapes.
encodings = [
    encoder.encode(ls1),
    encoder.encode(mp1),
    encoder.encode(ls2),
    encoder.encode(mp2)
]
distances = np.zeros((4, 4))
for i in range(4):
    for j in range(4):
        dd = encodings[i].sparse() - encodings[j].sparse()
        distances[i, j] = scipy.sparse.linalg.norm(dd)
distances

In [ ]:
import plotly.express as px
data=[[1, 25, 30, 50, 1], [20, 1, 60, 80, 30], [30, 60, 1, 5, 20]]
fig = px.imshow(distances,
                labels=dict(x="shape", y="shape", color="distance"),
                x=['ls1', 'mp1', 'ls2', 'mp2'],
                y=['ls1', 'mp1', 'ls2', 'mp2'],
               )
fig.update_xaxes(side="top")
fig.show()

In [ ]:
titles = [
    'cosine distance = %.4f' % c
    for c in cosines
]
fig = plotly.subplots.make_subplots(3, 1, subplot_titles=titles)

for i in range(len(x_offsets)):

    irow = i + 1
    
    # Draw the reference points.
    trace = Scatter(
        x=encoder.ref_x, y=encoder.ref_y, name='reference points', 
        mode='markers', marker={'color': 'gray', 'size': 5, 'symbol': 'cross'}
    )
    fig.append_trace(trace, irow, 1)

    # Draw the polygon.
    coords = g1.exterior.coords
    xx = [z[0] for z in coords]
    yy = [z[1] for z in coords]
    trace = Scatter(
        x=xx, y=yy, name='polygon', 
        mode='lines', marker={'color': 'red'}, fill='toself'
    )
    fig.append_trace(trace, irow, 1)

    # Draw the multipoint.
    xx = [z.xy[0][0] for z in geoms[i].geoms]
    yy = [z.xy[1][0] for z in geoms[i].geoms]
    trace = Scatter(
        x=xx, y=yy, name='cos dist = %.3f' % c, 
        mode='markers', marker={'color': 'blue', 'size': 12}
    )
    fig.append_trace(trace, irow, 1)

fig['layout']['width'] = 700
fig['layout']['height'] = 900
fig['layout']['xaxis1']['range'] = [0, 200]
fig['layout']['yaxis1']['range'] = [0, 100]

fig

## Cosine distance between encodings

In [ ]:
e1 = encoder.encode(g1).dense().ravel()
e2 = encoder.encode(g2).dense().ravel()

In [ ]:
dd = []
cc = []

for dx in np.linspace(0, 100, 50):
    g3 = shapely.affinity.translate(g2, xoff=dx)
    e3 = encoder.encode(g3).dense().ravel()
    c = scipy.spatial.distance.cosine(e1, e3)
    dd.append(dx)
    cc.append(c)

fig = plotly.subplots.make_subplots(1, 1)
trace = Scatter(x=dd, y=cc, mode='markers')
fig.append_trace(trace, 1, 1)
fig['layout']['xaxis1']['title'] = 'x offset'
fig['layout']['yaxis1']['title'] = 'cosine distance'
fig['layout']['width'] = 500
fig['layout']['height'] = 300

fig